In [ ]:
# # --- 0. CÀI ĐẶT & THƯ VIỆN ---
!pip install -q transformers sentence-transformers faiss-cpu

In [ ]:
import json
import pickle
import faiss

# === Đường dẫn (đổi theo dự án của bạn) ===
PATH_INDEX = "/kaggle/input/bge-bin/bge_origin"       # index đã build sẵn
PATH_CHUNK = "/kaggle/input/test-vlsp/chunk_corpus.json"  # corpus gốc
OUT_META   = "/kaggle/working/corpus_meta.pkl"        # nơi lưu meta mới

# 1) Load FAISS index để biết total vectors
index = faiss.read_index(PATH_INDEX)
N = index.ntotal
print("Index vectors:", N)

# 2) Load corpus (đảm bảo đúng thứ tự khi build index)
with open(PATH_CHUNK, "r", encoding="utf-8") as f:
    chunk_data = json.load(f)

# 3) Build meta
meta = []
for i, item in enumerate(chunk_data):
    # Bắt buộc phải có hai trường này
    if "aid" not in item or "chunk_id" not in item:
        raise ValueError(f"Item thứ {i} thiếu 'aid' hoặc 'chunk_id'")
    aid = item["aid"]
    cid = item["chunk_id"]
    meta.append((aid, cid))

print("Meta entries:", len(meta))

# 4) Kiểm tra khớp số lượng
if len(meta) != N:
    raise ValueError(
        f"Meta length ({len(meta)}) khác index.ntotal ({N}). "
        "Hãy kiểm tra lại thứ tự hoặc số chunk trong corpus."
    )

# 5) Lưu corpus_meta.pkl
with open(OUT_META, "wb") as f:
    pickle.dump(meta, f, protocol=pickle.HIGHEST_PROTOCOL)

print(f"✅ Đã lưu meta vào {OUT_META}")

In [ ]:
import json
import torch
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer

TOPK = 100

# ====== 1️⃣ Load file public test ======
with open("/kaggle/input/test-vlsp/test.json", "r", encoding="utf-8") as f:
    queries = json.load(f)

# ====== 2️⃣ Load FAISS index và metadata ======
index = faiss.read_index("/kaggle/input/bge-bin/bge_origin")
with open("/kaggle/working/corpus_meta.pkl", "rb") as f:
    meta = pickle.load(f)  # [(aid, chunk_id), ...]

# ====== 3️⃣ Load model BGE M3 ======
model = SentenceTransformer("/kaggle/input/bge-m3/pytorch/default/1/checkpoint-560")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ====== 4️⃣ Encode và search, xây dựng cấu trúc JSON ======
output = []
for q in queries:
    qid = q["qid"]
    question = q["question"]

    # Encode query
    q_emb = model.encode(question, normalize_embeddings=True, convert_to_numpy=True)

    # Search top-100
    D, I = index.search(np.array([q_emb]), k=TOPK)

    # Chuyển thành list các dict {"chunk_id": ..., "score": ...}
    top_chunks = [
        {"chunk_id": meta[idx][1], "score": float(D[0][i])}
        for i, idx in enumerate(I[0])
    ]

    output.append({
        "qid": qid,
        "top_chunks": top_chunks
    })

# ====== 5️⃣ Lưu kết quả ra file JSON ======
with open("bge_v0_results.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print("✅ Đã lưu kết quả")